<a href="https://colab.research.google.com/github/ayemish/sharkstack/blob/main/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip uninstall -y pinecone-client pinecone
!pip install pinecone langchain langchain-pinecone langchain-community langchain-text-splitters sentence-transformers pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
Found existing installation: pinecone 7.3.0
Uninstalling pinecone-7.3.0:
  Successfully uninstalled pinecone-7.3.0
  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
Using cached pinecone-7.3.0-py3-none-any.whl (587 kB)


In [2]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore


In [3]:
os.environ["PINECONE_API_KEY"] = "pcsk_6TDdRa_Eu5mZiJM1TX1cjTstQ9Ku8NxvYEFjXaFzfVkt63tgud7oJkqx2aHpgKJcKjGQSp"


In [10]:
loader = PyPDFLoader("book.pdf")
documents = loader.load()
print(f"Total pages loaded: {len(documents)}")


Total pages loaded: 594


In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n", " ", ""]
)
chunks = text_splitter.split_documents(documents)
print(f"Total chunks: {len(chunks)}")


Total chunks: 2812


In [12]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/tmp/ipython-input-102896465.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

index_name = "book-rag-chatbot"

# create index if not exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,   # embedding size for MiniLM-L6-v2
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# wait a bit before proceeding (index takes some seconds to initialize)


In [16]:
vectorstore = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embeddings,
    index_name=index_name
)


In [17]:
from transformers import pipeline

# load a small QA model
qa_model = pipeline("text2text-generation", model="google/flan-t5-small")

def rag_chatbot(query):
    # search in Pinecone
    results = vectorstore.similarity_search(query, k=3)
    context = " ".join([doc.page_content for doc in results])

    # instruction: only answer from context
    prompt = f"""Answer the question using ONLY the following context:

    {context}

    Question: {query}

    If the answer is not in the context, say: "I could not find it."
    """

    output = qa_model(prompt, max_length=256, do_sample=False)
    return output[0]['generated_text']

# Example
print(rag_chatbot("What is the main theme of chapter 5?"))


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (531 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


122 PART 2 /H17039DEVELOPING SUCCESSFUL BUSINESS IDEAS placed somewhere near the center of the page. A table of contents should follow the cover letter. It should list the sections and page numbers of the business plan and the appendices. Executive Summary The executive summary is a short overview of the entire business plan; it provides a busy reader with everything she needs to know about the new venture’s distinctive nature.


In [18]:
rag_chatbot("Summarize chapter 2.")
rag_chatbot("Who are the main characters?")
rag_chatbot("What happens at the end of the book?")


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'DEDICATION To my wife Jan. Thanks for your never-ending encouragement and support. Without you, this book would have never been possible. Also, thanks to all the student entrepreneurs who contributed to the chapter opening features in the book. Your stories are both insightful and inspiring. —Bruce R. Barringer To my family: I am so proud of each of you and so blessed by your perseverance and never-ending love and support. I know that sometimes it seems as though “we lose ourselves in work to do and bills to pay and that it’s a ride, ride, ride without much cover.” But you are always in my heart, a gift for which I remain deeply grateful. —R. Duane Ireland'

In [19]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 📚 Book Chatbot")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Ask a question about your book...")
    clear = gr.Button("Clear")

    def respond(message, chat_history):
        bot_reply = rag_chatbot(message)   # use your existing function
        chat_history.append((message, bot_reply))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()


/tmp/ipython-input-2824986204.py:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c9109f7ea3ae851c69.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
